In [ ]:
!pip install transformers --quiet

In [20]:
!id="1fdWdDm8LYw68kIZ6Z7xlT3Re8QqtLWis"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O mmcovid_en.csv && rm -rf /tmp/cookies.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2022-08-25 13:40:49--  https://docs.google.com/uc?export=download&confirm=&id=1fdWdDm8LYw68kIZ6Z7xlT3Re8QqtLWis
Resolving docs.google.com (docs.google.com)... 142.250.186.206, 2a00:1450:401b:80d::200e
Connecting to docs.google.com (docs.google.com)|142.250.186.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5s8se6csd5q1pm9jf4qfko82i3unu6tk/1661427600000/05536367513713019519/*/1fdWdDm8LYw68kIZ6Z7xlT3Re8QqtLWis?e=download&uuid=259db83d-7360-4079-86c4-7e69043a1bd3 [following]
--2022-08-25 13:40:50--  https://doc-0o-34-docs.googleusercontent

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("../../raw/mmcovid_en.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

    
X, y = data["text"].astype(str).values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 0    5304
1    2028
Name: label, dtype: int64
shape  
 (7332, 4)


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 2579.36it/s]

shapes X (7332,) y (7332,)


In [2]:
data.columns

Index(['Unnamed: 0', 'lang', 'label', 'text'], dtype='object')

In [21]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)
    
    def predict_proba(self, X=None):    
        return self.head.predict_proba(X)



using device: cuda


In [4]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|██████████████████████████████████████| 1833/1833 [00:15<00:00, 119.51it/s]


In [5]:
import pickle
# SAVE
with open('../../pickles/bertcls_embeddings/mmcovid_en_BERTEmbeddings.pickle', 'wb') as handle:
    pickle.dump(X_dstil_numpy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
import pickle
# LOAD
with open('../../pickles/bertcls_embeddings/mmcovid_en_BERTEmbeddings.pickle', 'rb') as handle:
    X_dstil_numpy=pickle.load(handle)

In [20]:
log_loss([1,1,0,0],[1,0.6,0,0])

0.12770640594149843

In [23]:
from sklearn.metrics import log_loss,accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def log_los_nonorm(x,y,average=False):
    return log_loss(x,y,normalize=True)

scores = {
    'Accuracy': {'func': accuracy_score},
    'Balanced Accuracy': {'func': balanced_accuracy_score},
    'F1': {'func': f1_score},
    'Precision': {'func': precision_score},
    'Recall': {'func': recall_score},
    'Log-likelihood': {'func': log_los_nonorm},
    #'G-mean': {'func': geometric_mean_score}
}

for score_name, score_dict in scores.items():
    scores[score_name]["list"] = []
    scores[score_name]["lab"] = []


for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])
    y_pred_proba = cls.predict_proba(X_dstil_numpy[test])
    
    for score_name, score_dict in scores.items():
        if score_name == "Log-likelihood":
            scorval=score_dict['func'](y[test], y_pred_proba)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
        elif score_name in ["F1","Precision","Recall"]:
            scorvaln = score_dict['func'](y[test], y_pred, average=None)
            score_dict['lab'].append(scorvaln)
            scorval = score_dict['func'](y[test], y_pred, average="weighted")
            score_dict['list'].append(scorval)
            print(score_name, scorval, scorvaln)  
        else:
            scorval=score_dict['func'](y[test], y_pred)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
    print()

Accuracy 0.9247135842880524
Balanced Accuracy 0.8763922728854856
F1 0.9220599580427447 [0.94979993 0.84950927]
Precision 0.9264372542276581 [0.91742797 0.95      ]
Recall 0.9247135842880524 [0.98453997 0.76824458]
Log-likelihood 0.18371681276618707

Accuracy 0.9293507910529187
Balanced Accuracy 0.8911706694512125
F1 0.9277083865208086 [0.95238095 0.86318014]
Precision 0.9293563378442766 [0.92931468 0.9294653 ]
Recall 0.9293507910529187 [0.97662142 0.80571992]
Log-likelihood 0.17411960234629925

Accuracy 0.928532460447354
Balanced Accuracy 0.8927369764473837
F1 0.9271025051232408 [0.95167835 0.86282723]
Precision 0.9281537697650686 [0.93140794 0.91964286]
Recall 0.928532460447354 [0.97285068 0.81262327]
Log-likelihood 0.17871460695466396

Accuracy 0.9222585924713584
Balanced Accuracy 0.8716498433693004
F1 0.9193339542281252 [0.94826647 0.84366429]
Precision 0.9242890503676223 [0.91424571 0.95055624]
Recall 0.9222585924713584 [0.98491704 0.75838264]
Log-likelihood 0.19705774746686677

Ac

In [25]:
numlabels = scores["F1"]["lab"][0].shape[0]
scores["F1"]["lab"][0].shape[0] 
head = "| %-20s | %-12s |" +  numlabels * " %-10s |" 
headv = ["Score", "Average"]
headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
row=head % tuple(headv)
#print("+"*len(row))
print(row)
#print("+"*len(row))

for score_name, score_dict in sorted(scores.items()) :
    if score_name == "Log-likelihood":
        headv = [score_name, np.mean(score_dict['list'])*1, np.std(score_dict['list'])*1]
    else:
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
    
    for i in range(numlabels):
        if score_name == "Log-likelihood":
            head = "| %-20s | %3.2f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
        elif score_name in ["F1","Precision","Recall"]:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels* " %4.1f ± %3.1f |"
            vals = [v[i] for v in scores[score_name]["lab"]]
            headv.append(np.mean(vals)*100)
            headv.append(np.std(vals)*100)
        else:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
    print(head % tuple(headv))
    
#print("+"*len(row))

| Score                | Average      | Kat_1      | Kat_2      |
| Accuracy             | 92.6 ± 0.226 | -          | -          |
| Balanced Accuracy    | 88.3 ± 0.681 | -          | -          |
| F1                   | 92.4 ± 0.268 | 95.1 ± 0.1 | 85.5 ± 0.6 |
| Log-likelihood       | 0.18 ± 0.007 | -          | -          |
| Precision            | 92.7 ± 0.169 | 92.3 ± 0.6 | 93.9 ± 1.1 |
| Recall               | 92.6 ± 0.226 | 98.0 ± 0.4 | 78.5 ± 1.7 |
